In [ ]:
from river.datasets import synth
from clustream import CluStream
from river import drift
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd
from scipy.spatial import distance_matrix

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA
import itertools




In [ ]:
win = 500
mc = 30
speed = 100

In [ ]:
ds = pd.read_csv('./datasets/rbf speed 100l/rbf_speed_100L.csv')
#ds = pd.read_csv('./datasets/rbf create delete 2/rbf_create_delete_2.csv')
df = ds.copy()
y = df.pop('.class')
df.head()

In [ ]:
model = CluStream(n_macro_clusters=3,max_micro_clusters=mc,micro_cluster_r_factor=0.01,time_window=500,time_gap=500)
test = drift.PageHinkley(min_instances=10,delta=0.04899,threshold=0.15,alpha=1,mode='up')

In [ ]:
fig ,ax = plt.subplots(2,3,figsize=(10,6),sharex=True, sharey= True)
it = itertools.product((0,1,2),(0,1,2))
colors = {1 : 'blue', 2 : 'red', 3 : 'green', 4 : 'purple' , 5 : 'cyan', 6 : 'orange' , 7 : 'yellow', 8 : 'black' }
i = 0
j = 0
go = True
dist = []
mean_dist = []
drift_inst = []
for x,y in stream.iter_pandas(df,y):
    model = model.learn_one(x)
    i +=1

    if model._initialized :
        if go :
            model.offline_cluster()
            go = False
        test = test.update(model.dist_center(x))
        if test.drift_detected :
             print(f'Drift detected at instance: {i} ')
             drift_inst.append([i,mean_dist[-1]])
        dist.append(model.dist_center(x))
        if j % 100 == 0:
            print(np.mean(dist))
            mean_dist.append(np.mean(dist))
            dist = []
        j += 1

    if i % 500 == 0 :

        k,m = next(it)
        centroids = model.get_micro_clusters()
        grouped = ds[i:i+250].groupby('.class')
        for key,group in grouped :
            group.plot(ax=ax[k,m],x='X1',y='X2',kind="scatter",label = key,color=colors[key],title=f'De :  {i} à : {i + 250}')
        ax[k,m].scatter(centroids[:,0],centroids[:,1],marker="x",c='black')
        # ax[k,m].scatter(model.centers[:,0],model.centers[:,1],marker='o',c='white')
    
    if i ==3000 :
        break
    
    
        
fig.suptitle(f'RBF Vitesse {speed} , Fênetre {win}, MC : {mc}', fontsize=12)
fig.savefig(f'window_{win}_{mc}mc')

In [ ]:
drift_inst = np.array(drift_inst)

In [ ]:
fig1 = plt.figure('Fig 2')
plt.plot(np.arange(100,3100,100),mean_dist)
plt.scatter(drift_inst[:,0],drift_inst[:,1],c='red',label='Points de drift')
plt.xlabel('Instance')
plt.ylabel('Distance min du centroid')
plt.title('Variation de la distance moyenne au centroids ')
plt.legend()

fig1.suptitle(f'RBF Vitesse {speed} , Fênetre {win}, MC : {mc}', fontsize=12)
fig1.savefig(f'drift_{win}_{mc}mc')